In [1]:
import datetime as dt
from os.path import exists
from pathlib import Path
import numpy as np
import pandas as pd
import yfinance as yf
from yahooquery import Ticker

import src.tools.functions as f0

pd.set_option('display.max_columns', None)
np.random.seed(42)

day1 = '2022-07-28'
month1 = str(day1)[:7]
year1 = str(day1)[:4]
bulk_data_file = Path(f"/home/gdp/hot_box/larrys_law/data/finviz/{month1}/{day1}/finviz.csv")

In [13]:
def larry_1():
    mkt_cap_min = 1000000000
    mkt_cap_max = 2340000000

    data_0 = pd.read_csv(bulk_data_file)
    data_0 = f0.clean_sort(data_0, mkt_cap_min, mkt_cap_max)

    col_1 = data_0.pop('market_cap')
    col_2 = data_0.pop('ticker')
    data_0.insert(0, 'market_cap', col_1) 
    data_0.insert(0, 'ticker', col_2) 
    
    print(data_0.shape)
    return data_0.set_index('company')

In [14]:
data = larry_1()
data.head()

(791, 67)


,ticker,market_cap,sector,industry,country,dividend_yield,payout_ratio,insider_ownership,insider_transactions,institutional_ownership,institutional_transactions,float_short,return_on_assets,return_on_equity,return_on_investment,gross_margin,operating_margin,profit_margin,performance_half_year,performance_year,performance_ytd,volatility_week,volatility_month,gap,eps_growth_this_year,eps_growth_next_year,eps_growth_past_5_years,eps_growth_next_5_years,sales_growth_past_5_years,eps_growth_quarter_over_quarter,sales_growth_quarter_over_quarter,performance_week,performance_month,performance_quarter,no,pe,forward_pe,peg,ps,pb,pcash,pfree_cash_flow,eps_ttm,current_ratio,quick_ratio,lt_debtequity,total_debtequity,shares_outstanding,shares_float,short_ratio,beta,average_true_range,relative_strength_index_14,analyst_recom,average_volume,relative_volume,price,volume,target_price,sma_20,sma_50,sma_200,high_50_day,low_50_day,high_52_week,low_52_week
company,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ares Acquisition Corporation,AAC,"$1,231,220,000.00",Financial,Shell Companies,USA,NaN,0.000,NaN,NaN,0.886,0.0012,0.0053,0.013,0.013,NaN,NaN,NaN,NaN,0.0144,0.0113,0.0123,0.0021,0.0020,0.0000,NaN,NaN,NaN,NaN,NaN,-0.163,NaN,0.0010,0.0020,0.0020,4.0,98.60,NaN,NaN,NaN,1.30,2462.44,NaN,0.10,0.3,0.3,0.00,0.00,125.00,100.00,1.37,NaN,0.02,62.24,NaN,387.26,0.57,9.87,218951.0,NaN,9.847351,9.828719,9.795554,9.879880,9.789724,9.879880,9.660370
"American Assets Trust, Inc.",AAT,"$1,814,000,000.00",Real Estate,REIT - Diversified,USA,0.0429,1.913,0.005,NaN,0.969,-0.0015,0.0167,0.012,0.030,0.033,0.655,0.275,0.096,-0.1593,-0.1897,-0.2044,0.0243,0.0218,0.0040,0.023,0.1475,-0.080,0.0300,0.050,7.366,0.208,-0.0090,-0.0145,-0.1671,19.0,47.62,42.66,15.87,4.61,1.45,24.65,NaN,0.63,NaN,NaN,1.33,1.33,60.04,59.85,4.04,1.14,0.73,51.98,3.2,247.83,0.81,30.27,199745.0,35.38,29.740617,30.815433,35.300292,34.280861,28.379899,40.828163,28.379899
"Atlas Air Worldwide Holdings, Inc.",AAWW,"$1,972,910,000.00",Industrials,Airports & Air Services,USA,NaN,0.000,0.013,-0.0270,NaN,0.0013,0.1209,0.077,0.181,0.108,0.580,0.172,0.115,-0.0886,0.0799,-0.2430,0.0294,0.0324,0.0059,0.197,-0.1556,0.569,0.1000,0.170,-0.221,0.204,0.0417,0.0903,0.0512,22.0,4.63,5.03,0.46,0.47,0.74,2.70,2.08,15.38,0.8,0.8,0.58,0.81,28.85,27.82,7.68,1.26,2.12,73.43,2.5,438.47,1.27,73.19,558362.0,99.33,63.921397,65.782851,77.482532,72.811381,58.702278,97.133378,58.702278
"Accel Entertainment, Inc.",ACEL,"$1,079,370,000.00",Consumer Cyclical,Gambling,USA,NaN,0.000,0.059,-0.0370,0.671,0.0007,0.0347,0.075,0.286,0.108,0.327,0.103,0.058,-0.0559,0.0439,-0.1052,0.0293,0.0295,0.0077,0.293,0.1594,NaN,0.2174,NaN,9.572,0.339,0.0402,0.0649,-0.0102,57.0,23.92,10.40,1.10,1.38,6.51,4.75,14.07,0.49,3.5,3.5,1.94,2.05,92.99,58.74,9.04,1.02,0.33,71.91,2.2,225.55,1.15,12.05,260107.0,15.50,11.076386,10.772394,12.159435,11.714952,9.780050,14.095216,9.780050
"Axcelis Technologies, Inc.",ACLS,"$2,313,530,000.00",Technology,Semiconductor Equipment & Materials,USA,NaN,0.000,0.028,-0.0361,0.851,0.0025,0.0346,0.170,0.235,0.180,0.436,0.213,0.169,0.1281,0.8731,-0.1011,0.0440,0.0538,0.0015,0.966,0.0873,0.519,0.2000,0.199,1.575,0.533,0.0352,0.1492,0.2427,76.0,18.45,13.94,0.92,3.16,3.98,7.79,15.22,3.63,4.4,3.0,0.08,0.08,33.24,32.33,3.07,1.69,3.10,68.51,1.8,363.99,0.68,68.46,245904.0,82.83,56.648738,57.428068,61.814898,67.997616,46.410413,83.732877,36.080953


In [15]:
def fte(stocks):
    fte_list = []
    for stock in stocks:
        try:
            yqT = Ticker(stock)
            x = pd.DataFrame(yqT.asset_profile)
            y = float(x[x.index == 'fullTimeEmployees'][f"{stock}"][0])
            fte_list.append(y)
        except Exception:
            fte_list.append(0)
    return fte_list


def revenue(stocks):
    list1 = []
    for stock in stocks:
        try:
            yqT = Ticker(stock)
            x = yqT.income_statement(frequency='a').iloc[-1]['TotalRevenue']
            list1.append(round(float(x),2))
        except Exception:
            list1.append(0)
    return list1    

In [16]:
stock_list = list(data['ticker'])

fte_list = fte(stock_list)
revenue_list = revenue(stock_list)

2022-08-09 11:46:34.174 WARNING urllib3.connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5faa80b350>: Failed to establish a new connection: [Errno 101] Network is unreachable')': /v10/finance/quoteSummary/AMWL?modules=assetProfile&formatted=false&lang=en-US&region=US&corsDomain=finance.yahoo.com
2022-08-09 11:49:53.490 WARNING urllib3.connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5fab193790>: Failed to establish a new connection: [Errno 101] Network is unreachable')': /v10/finance/quoteSummary/GBX?modules=assetProfile&formatted=false&lang=en-US&region=US&corsDomain=finance.yahoo.com
2022-08-09 11:51:31.042 WARNING urllib3.connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None

In [17]:
df = pd.DataFrame(data.copy())

df['fte'] = fte_list
df['revenue'] = revenue_list

col_1 = data_0.pop('revenue')
col_2 = data_0.pop('fte')
data_0.insert(0, 'revenue', col_1) 
data_0.insert(0, 'fte', col_2)

df.head()

,ticker,market_cap,sector,industry,country,dividend_yield,payout_ratio,insider_ownership,insider_transactions,institutional_ownership,institutional_transactions,float_short,return_on_assets,return_on_equity,return_on_investment,gross_margin,operating_margin,profit_margin,performance_half_year,performance_year,performance_ytd,volatility_week,volatility_month,gap,eps_growth_this_year,eps_growth_next_year,eps_growth_past_5_years,eps_growth_next_5_years,sales_growth_past_5_years,eps_growth_quarter_over_quarter,sales_growth_quarter_over_quarter,performance_week,performance_month,performance_quarter,no,pe,forward_pe,peg,ps,pb,pcash,pfree_cash_flow,eps_ttm,current_ratio,quick_ratio,lt_debtequity,total_debtequity,shares_outstanding,shares_float,short_ratio,beta,average_true_range,relative_strength_index_14,analyst_recom,average_volume,relative_volume,price,volume,target_price,sma_20,sma_50,sma_200,high_50_day,low_50_day,high_52_week,low_52_week,fte,revenue
company,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Ares Acquisition Corporation,AAC,"$1,231,220,000.00",Financial,Shell Companies,USA,NaN,0.000,NaN,NaN,0.886,0.0012,0.0053,0.013,0.013,NaN,NaN,NaN,NaN,0.0144,0.0113,0.0123,0.0021,0.0020,0.0000,NaN,NaN,NaN,NaN,NaN,-0.163,NaN,0.0010,0.0020,0.0020,4.0,98.60,NaN,NaN,NaN,1.30,2462.44,NaN,0.10,0.3,0.3,0.00,0.00,125.00,100.00,1.37,NaN,0.02,62.24,NaN,387.26,0.57,9.87,218951.0,NaN,9.847351,9.828719,9.795554,9.879880,9.789724,9.879880,9.660370,0.0,0.000000e+00
"American Assets Trust, Inc.",AAT,"$1,814,000,000.00",Real Estate,REIT - Diversified,USA,0.0429,1.913,0.005,NaN,0.969,-0.0015,0.0167,0.012,0.030,0.033,0.655,0.275,0.096,-0.1593,-0.1897,-0.2044,0.0243,0.0218,0.0040,0.023,0.1475,-0.080,0.0300,0.050,7.366,0.208,-0.0090,-0.0145,-0.1671,19.0,47.62,42.66,15.87,4.61,1.45,24.65,NaN,0.63,NaN,NaN,1.33,1.33,60.04,59.85,4.04,1.14,0.73,51.98,3.2,247.83,0.81,30.27,199745.0,35.38,29.740617,30.815433,35.300292,34.280861,28.379899,40.828163,28.379899,206.0,4.056580e+08
"Atlas Air Worldwide Holdings, Inc.",AAWW,"$1,972,910,000.00",Industrials,Airports & Air Services,USA,NaN,0.000,0.013,-0.0270,NaN,0.0013,0.1209,0.077,0.181,0.108,0.580,0.172,0.115,-0.0886,0.0799,-0.2430,0.0294,0.0324,0.0059,0.197,-0.1556,0.569,0.1000,0.170,-0.221,0.204,0.0417,0.0903,0.0512,22.0,4.63,5.03,0.46,0.47,0.74,2.70,2.08,15.38,0.8,0.8,0.58,0.81,28.85,27.82,7.68,1.26,2.12,73.43,2.5,438.47,1.27,73.19,558362.0,99.33,63.921397,65.782851,77.482532,72.811381,58.702278,97.133378,58.702278,4056.0,4.396224e+09
"Accel Entertainment, Inc.",ACEL,"$1,079,370,000.00",Consumer Cyclical,Gambling,USA,NaN,0.000,0.059,-0.0370,0.671,0.0007,0.0347,0.075,0.286,0.108,0.327,0.103,0.058,-0.0559,0.0439,-0.1052,0.0293,0.0295,0.0077,0.293,0.1594,NaN,0.2174,NaN,9.572,0.339,0.0402,0.0649,-0.0102,57.0,23.92,10.40,1.10,1.38,6.51,4.75,14.07,0.49,3.5,3.5,1.94,2.05,92.99,58.74,9.04,1.02,0.33,71.91,2.2,225.55,1.15,12.05,260107.0,15.50,11.076386,10.772394,12.159435,11.714952,9.780050,14.095216,9.780050,900.0,7.845290e+08
"Axcelis Technologies, Inc.",ACLS,"$2,313,530,000.00",Technology,Semiconductor Equipment & Materials,USA,NaN,0.000,0.028,-0.0361,0.851,0.0025,0.0346,0.170,0.235,0.180,0.436,0.213,0.169,0.1281,0.8731,-0.1011,0.0440,0.0538,0.0015,0.966,0.0873,0.519,0.2000,0.199,1.575,0.533,0.0352,0.1492,0.2427,76.0,18.45,13.94,0.92,3.16,3.98,7.79,15.22,3.63,4.4,3.0,0.08,0.08,33.24,32.33,3.07,1.69,3.10,68.51,1.8,363.99,0.68,68.46,245904.0,82.83,56.648738,57.428068,61.814898,67.997616,46.410413,83.732877,36.080953,1122.0,8.071500e+08


In [ ]:
df['rev_per_fte'] = df['revenue'] / df['fte']

# df.replace([np.inf, -np.inf], 0, inplace=True)
# df = df.sort_values('rev_per_fte', ascending=False).fillna(0.0)

df

In [ ]:
df['fte'] = df['fte'].apply(lambda x: f"{x:,.2f}")
df['market_cap'] = df['market_cap'].apply(lambda x: f"${x*1000000:,.2f}")
df['revenue'] = df['revenue'].apply(lambda x: f"${x:,.2f}")
df['rev_per_fte'] = df['rev_per_fte'].apply(lambda x: f"${x:,.2f}")
df

In [ ]:
df.to_csv('larry.csv')